# Lab 4: Radiometric calibration, spectral indices, and transformations

**Purpose:** The purpose of this lab is to walk through radiometric calibration of Landsat data as well as give you a tour of spectral indices that can be used to enhance phenomena of interest in remotely sensed images.  You will be introduced to methods for creating vegetation, water, snow, bare and burned area indices.  You will explore spectral unmixing.  At the completion of the lab, you will be able to implement spectral indices and transforms to accentuate the information of interest in your study area.


In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Radiometric Calibration - Landsat

We have used Landsat data with a couple of different units (Digital Numbers, Top-of-atmosphere reflectance, and surface reflectance), however, the imagery is typically stored as DNs.  The example will walk though the process to convert DN values into top-of-atmophere reflectance.

### Top-of-atmopshere Radiance

To convert DN values into at-sensor radiance units in $Watts \cdot m^{-2} \cdot sr^{-1} \cdot \mu m^{-1}$, use a linear equation of the form:

$L_\lambda = a_\lambda \cdot DN_\lambda + b_\lambda$

Note that every term is indexed by lamda ($\lambda$, the symbol for wavelength) because the coefficients are different in each band.  See [Chander et al. (2009)](https://doi.org/10.1016/j.rse.2009.01.007) for details on this linear transformation between DN and radiance.

**Note:** At-sensor and top-of-atmosphere are sometimes used interchagebly.

In [ ]:
# load in a Landsat image based on 
image_id = "LANDSAT/LC08/C02/T1/LC08_044034_20141012"

# select only the first 7 bands in the image which 
# are the visible to middle infrared bands
image = ee.Image(image_id).select("B[1-7]")

In [ ]:
# check the band names
image.bandNames().getInfo()

In [ ]:
image.getInfo()

In [ ]:
# define lists of the property names that represent the 
# scale and offset for calculating radiance
rad_add_names = [f"RADIANCE_ADD_BAND_{i}" for i in range(1,8)]
rad_mult_names = [f"RADIANCE_MULT_BAND_{i}" for i in range(1,8)]

In [ ]:
# extract out the scale and offset metadata values for the bands
rad_add_vals = image.toDictionary(rad_add_names)
rad_mult_vals = image.toDictionary(rad_mult_names)

# convert the values to an 7-band image
rad_add_img = rad_add_vals.toImage()
rad_mult_img = rad_mult_vals.toImage()

In [ ]:
# apply the scale and offset factors to 
# convert DN to TOA radiance
radiance = image.multiply(rad_mult_img).add(rad_add_img)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B4","B3","B2"], "min": 5000, "max": 15000, "gamma": 1.3}, 'DN values');
Map.addLayer(radiance,{"bands":"B4,B3,B2", "min":5, "max":100, "gamma":1.3}, "TOA Radiance")

Map.addLayerControl()

Map

### Top-of-atmopshere Reflectance

Now that we have TOA Radiance we would like to convert these to reflectance values. To do so, we can apply to 

$\rho_\lambda = \frac{\pi \cdot L_\lambda \cdot d^2}{ESUN_\lambda \cdot cos\theta_{sz}}$

There are a few more things we would need to extract to caculate

In [ ]:
# create and image for pi
pi = ee.Image(math.pi)

# extract out the earth sun distance and create an image
d = image.metadata("EARTH_SUN_DISTANCE")

# extract out the solar elevation angle and create an image
# convert from degrees to radians
se = image.metadata("SUN_ELEVATION").multiply(pi.divide(180))

USGS and NASA decided not to publish ESUN values because they are not required for conversion to reflectance any more but we can still calculate them from the radiance and reflectance scale factors.



In [ ]:
# create a list of 
ref_mult_names = [f"REFLECTANCE_MULT_BAND_{i}" for i in range(1,8)]
ref_mult_vals = image.toDictionary(ref_mult_names)
ref_mult_img = ref_mult_vals.toImage()

In [ ]:
esun = ee.Image().expression("pi * (d**2) * (radb / refb)",{
    "pi": pi,
    "d": d,
    "radb": rad_mult_img,
    "refb": ref_mult_img
})

In [ ]:
reflectance = ee.Image().expression("(pi*rad*(d**2))/(esun * sin(se))",{
    "pi": pi,
    "rad": radiance,
    "d": d,
    "esun": esun,
    "se": se
})

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["B4","B3","B2"], "min": 5000, "max": 15000, "gamma": 1.3}, 'DN values');
Map.addLayer(radiance,{"bands":"B4,B3,B2", "min":5, "max":100, "gamma":1.3}, "TOA Radiance")
Map.addLayer(reflectance,{"bands":"B4,B3,B2", "min":0, "max":0.33, "gamma":1.3}, "TOA Reflectance")


Map.addLayerControl()

Map

### Surface reflectance

To calculate surface reflectance requires a much more complex process. We will not go through calculating surface reflectance but rather import the data directly to visualize.

In [ ]:
# read in surface reflectance image
# rescale to surface reflectance values
# select bands 1-7 
sr = (
    ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_044034_20141012")
    .multiply(2.75e-05).add(-0.2)
    .select("SR_B[1-7]")
)

### Spectra from different units

The conversion from top-of-atmosphere radiance to surface reflectance is not to just make pretty pictures, we are adjusting the physical values to represent what we would measure on the ground. The results 

In [ ]:
%pylab inline

In [ ]:
water = ee.Geometry.Point(-122.6787, 37.5181)
forest = ee.Geometry.Point(-122.6715, 37.9494)
urban = ee.Geometry.Point(-122.41712, 37.75641)
ag = ee.Geometry.Point(-121.50267, 37.85451)

pts = ee.FeatureCollection([water,forest,urban,ag])

In [ ]:
def plot_spectra(img, units):
    spectra_dict = img.reduceRegions(pts, ee.Reducer.mean(), 30).getInfo()

    f,ax = plt.subplots(figsize=(10,5))

    for i,f in enumerate(spectra_dict["features"]):
        spectra = list(f["properties"].values())
        ax.plot(wavelengths, spectra, color=colors[i], marker="o",label=feature_names[i],lw=1.5)

    ax.legend()

    ax.set_xlabel("Wavelength [$\mu m$]")
    ax.set_ylabel(units)
    return

colors = ["C0","C2","C1","C4"]
wavelengths = [0.44, 0.48,0.56,0.655,0.865,1.61,2.20]
feature_names = ["Water","Forest", "Urban", "Agriculture"]

In [ ]:
plot_spectra(radiance,"Radiance")

In [ ]:
plot_spectra(reflectance,"TOA Reflectance")

In [ ]:
plot_spectra(sr,"Surface Reflectance")

Notice how the spectral curves of the different land surface features change with each processed image. What do these changes mean? Think of the physical process we are compensating for at each step.

## Spectral indices

Now that we have a good understanding of why and how to create surface reflectance data, we will explore what to do with it. Spectral indices are based on the fact that reflectance spectra of different land covers are different (as seen from the plots above).  The indices are designed to exploit these differences to accentuate particular land cover types. 

From thea above plots we can observe that the land covers are separable at one or more wavelengths.  Note, in particular, that vegetation curves have relatively high reflectance in the NIR range, where radiant energy is scattered by cell walls ([Bowker et al. 1985](http://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19850022138.pdf)).  Also note that vegetation has low reflectance in the red range, where radiant energy is [absorbed by chlorophyll](https://en.wikipedia.org/wiki/Chlorophyll#/media/File:Chlorophyll_ab_spectra-en.svg).  These observations motivate the formulation of vegetation indices, such as the Normalized Difference Vegetation Index (NDVI).

In [ ]:
# before beginning set out surface reflectance image to the `image` variable
# for readability
image = sr

### NDVI

The Normalized Difference Vegetation Index (NDVI) has a [long history](https://en.wikipedia.org/wiki/Normalized_Difference_Vegetation_Index) in remote sensing.  The typical formulation is

$NDVI = \frac{(NIR - red)}{(NIR + red)}$

Where NIR and red refer to reflectance, radiance or DN at the respective wavelength.

We are going to use the built-in EE function `.normalizedDifference()` to calculate NDVI:

In [ ]:
ndvi = image.normalizedDifference(["SR_B5","SR_B4"])

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(ndvi,{"min":0,"max":1, "palette":cmaps.get_palette("Greens")}, "NDVI")


Map.addLayerControl()

Map

Do you notice anything missing from the image? Why would there be missing pixels?

In [ ]:
# create our own normalized difference function to prevent missing pixels
def normalized_difference(img,bands):
    b1 = img.select(bands[0])
    b2 = img.select(bands[1])

    return b1.subtract(b2).divide(b1.add(b2)).rename("nd").clamp(-1,1)

### NDWI

The Normalized Difference Water Index (NDWI) was developed by [Gao (1996)](http://www.sciencedirect.com/science/article/pii/S0034425796000673) as an index of vegetation water content:

$NDWI = \frac{(NIR - SWIR)}{(NIR + SWIR)}$


In [ ]:
ndwi = normalized_difference(image, ["SR_B5","SR_B6"])

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(ndwi,{"min":-1,"max":1, "palette":cmaps.get_palette("Blues")}, "NDWI")


Map.addLayerControl()

Map

### MNDWI

The modified Normalized Difference Water Index (mNDWI) was developed by [Xu 2006](https://www.tandfonline.com/doi/abs/10.1080/01431160600589179) to better distigush open water as opposed to vegetation water content which is what NDWI highlights. It takes the form of the following:

$MNDWI = \frac{(green - SWIR1)}{(green + SWIR1)}$

In [ ]:
mndwi = normalized_difference(image,["SR_B3","SR_B6"])

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(mndwi,{"min":-1,"max":1, "palette":cmaps.get_palette("Blues")}, "mNDWI")


Map.addLayerControl()

Map

It should be noted that there is *another* normalized difference water index developed in 1996 by [McFeeters (1996)](http://www.tandfonline.com/doi/abs/10.1080/01431169608948714#.VkThFHyrTlM) called the Normalized Difference Water Body Index (NDWBI) which uses the green and NIR bands. Be aware that there are multiple indices.

### NDBI

The Normalized Difference Bare Index (NDBI) was developed by [Zha et al. (2003)](http://www.tandfonline.com/doi/abs/10.1080/01431160304987) to aid in the differentiation of urban areas:

$NDBI = \frac{SWIR - NIR}{SWIR + NIR}$

**Note:** Note that NDBI is the negative of NDWI.

In [ ]:
ndbi = normalized_difference(image, ["SR_B6","SR_B5"])

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(ndbi,{"min":-1,"max":0.5, "palette":cmaps.get_palette("Oranges")}, "NDBI")


Map.addLayerControl()

Map

### BAI

The Burned Area Index (BAI) was developed by [Chuvieco et al. (2002)](http://www.tandfonline.com/doi/abs/10.1080/01431160210153129) to assist in the delineation of burn scars and assessment of burn severity.  It is based on the spectral distance to charcoal reflectance.  To examine burn indices, load an image from 2013 showing the [Rim fire](https://en.wikipedia.org/wiki/Rim_Fire) in the Sierra Nevadas

In [ ]:
lc8_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [ ]:
burn_image = ee.Image(
    lc8_sr
    .filterBounds(ee.Geometry.Point(-120.083, 37.850))
    .filterDate('2013-08-17', '2013-09-27')
    .sort('CLOUD_COVER')
    .first()
    .multiply(2.75e-05).add(-0.2) # rescale
)


In [ ]:
 bai = burn_image.expression(
    '1.0 / ((0.1 - RED)**2 + (0.06 - NIR)**2)', {
      'NIR': burn_image.select('SR_B5'),
      'RED': burn_image.select('SR_B4'),
})


In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(burn_image, 10)

Map.addLayer(burn_image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(bai,{"min":0,"max":400, "palette":cmaps.get_palette("Oranges")}, "BAI", False)


Map.addLayerControl()

Map

### NDSI

The Normalized Difference Snow Index (NDSI) was designed to estimate the amount of a pixel covered in snow ([Riggs et al. 1994](http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=399618&tag=1)):

$NDSI = \frac{green - SWIR}{green + SWIR}$

Do you notice anything interesting about this index compared to the others?


In [ ]:
snow_image = ee.Image(
    lc8_sr
    .filterBounds(ee.Geometry.Point(-120.0421, 39.1002))
    .filterDate('2013-11-01', '2014-05-01')
    .sort('CLOUD_COVER')
    .first()
    .multiply(2.75e-05).add(-0.2) # rescale
)

In [ ]:
ndsi = normalized_difference(snow_image,["SR_B3","SR_B6"])

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(snow_image, 10)

Map.addLayer(snow_image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(ndsi,{"min":-1,"max":1, "palette":cmaps.get_palette("cubehelix")}, "NDSI")


Map.addLayerControl()

Map

## Spectral unmixing

The [linear spectral mixing model](http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=974727&tag=1) is based on the assumption that each pixel is a mixture of "pure" spectra.  The pure spectra, called endmembers, are from land cover classes such as water, bare land, vegetation.  The goal is to solve the following equation for f, the Px1 vector of endmember fractions in the pixel: 

$Sf = p$

where **S** is a *BxP* matrix in which the columns are *P* pure endmember spectra (known) and **p** is the *Bx1* pixel vector when there are B bands (known).  In this example, B=6.

Consider the following spectral curves:

![Mixed Spectra](https://imgur.com/oCdDPrW.png)

In this case Water = 50% of the pixel, Dense Veg. = 40% of the pixel, and Sparse Veg. = 10% of pixel. By compining the weights at each band we get the spectral curve in black. The black spectral curve is what the satellite observes and we can estimate the percent coverage of each land cover type based on what we expect from a "pure pixel".

In [ ]:
unmix_image = image.select(['SR_B[2-7]'])

The first step is to get the endmember spectra.  There are algorithms available to estimate the "pure" pixel spectra (such as the Dynamic Nearest Neighbor Search algorithm). However, for this case we we hard code our endmember spectra:


In [ ]:
# define expected spectra for 
water = [0.032,0.055,0.037,0.001,0.001,0.002]
urban = [0.18,0.24,0.26,0.265,0.315,0.315]
veg = [0.01,0.019,0.015,0.168,0.069,0.027]

In [ ]:
# apply the unmixing and force the results to be 0-1
fractions = unmix_image.unmix([urban,veg,water],sumToOne=True,nonNegative=True)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(image, 10)

Map.addLayer(image, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.33, "gamma": 1.3}, 'Surface Reflectance');
Map.addLayer(fractions,{"min":0,"max":1,}, "Unmixed Fractions")


Map.addLayerControl()

Map